In [85]:
# Dependencies
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
merged_data_filepath = "../DataFrames/merged_set_data.csv"

merged_table = pd.read_csv(merged_data_filepath)



In [86]:
#Import the merged Pollution average and mortality count dataframe
merged_table = merged_table.drop(columns=['Unnamed: 0'])
merged_table.head()

,Year,NO2 Mean,O3 Mean,SO2 Mean,CO Mean,Year Average,Country,Variable,Unit,Value
0,2000,0.017433,0.024376,0.003649,0.557789,0.150812,United States,Mortality from exposure to outdoor PM2.5,Per 1 000 000 inhabitants,362.2790
1,2001,0.017600,0.025388,0.003133,0.509119,0.138810,United States,Mortality from exposure to outdoor PM2.5,Per 1 000 000 inhabitants,361.3241
2,2002,0.016387,0.026818,0.002744,0.500990,0.136735,United States,Mortality from exposure to outdoor PM2.5,Per 1 000 000 inhabitants,359.6222
3,2003,0.015865,0.025521,0.002865,0.492720,0.134243,United States,Mortality from exposure to outdoor PM2.5,Per 1 000 000 inhabitants,355.1779
4,2004,0.014968,0.024914,0.002855,0.448132,0.122717,United States,Mortality from exposure to outdoor PM2.5,Per 1 000 000 inhabitants,342.3452


In [87]:
#Create dictionary which we will use to acquire state abbrevation strings
#The value of this dictionary is mainly to provide us with the reverse dictionary which we create later
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
#create extra key value pairs for DC and PR
us_state_abbrev['Washington, DC'] = 'DC'
us_state_abbrev['Puerto Rico'] = 'PR'

In [88]:
#Create a reverse of the State abbrevation dictionary which we will use to convert abbrevations to State values 
reverse_us_state_abbrev = {}
for key, val in us_state_abbrev.items():
    reverse_us_state_abbrev[val] = key


In [89]:
#create a dictionary matching state 'keys' to corresponding FIPs codes 'values'
#FIPs are importnt since the Census api wrapper requires FIPs in order to acquire information on specific states
State_to_FIPs_dict = {'AK':'02', 'MS':'28',
'AL':'01', 'MT':'30',
'AR':'05', 'NC':'37',
'AS':'60', 'ND':'38',
'AZ':'04', 'NE':'31',
'CA':'06',  'NH':'33',
'CO':'08', 'NJ':'34',
'CT':'09',  'NM':'35',
'DC':'11', 'NV':'32',
'DE':'10', 'NY':'36',
'FL':'12', 'OH':'39',
'GA':'13', 'OK':'40',
'GU':'66', 'OR':'41',
'HI':'15', 'PA':'42',
'IA':'19', 'PR':'72',
'ID':'16', 'RI':'44',
'IL':'17', 'SC':'45',
'IN':'18', 'SD':'46',
'KS':'20', 'TN':'47',
'KY':'21', 'TX':'48',
'LA':'22', 'UT':'49',
'MA':'25', 'VA':'51',
'MD':'24', 'VI':'78',
'ME':'23', 'VT':'50',
'MI':'26', 'WA':'53',
'MN':'27', 'WI':'55',
'MO':'29', 'WV':'54',
'WY':'56'}

In [90]:
#create a reverse dictionary matching FIPs codes 'keys' to corresponding state 'values'
FIPs_to_State_dict = {}
for key, val in State_to_FIPs_dict.items():
    FIPs_to_State_dict[val] = key

In [91]:
#Create a list of States
State_list = [key for key in us_state_abbrev]

In [92]:
#Now we use a for loop to merge data sets

#Create an empty dataframe which we will append data to
State_Pop_by_year = pd.DataFrame()

#Run a for loop on the series of years in the merge_table dataframe that we imported above
for year in merged_table['Year']:
#use a try and except loop to account for potential api requests failing due to missing info
    try:
        
#set variable c equal to the census api wrapper with 'api key' and 'year' arguments
        c = Census(api_key, year=year)
#use request data using get command
        census_data = c.acs5.get(('STATE', 'B01003_001E'), {'for': 'state:*'})
#note census_data will be a list od dictionaries, which we can use to create a dataframe
        census_pd = pd.DataFrame(census_data)
#create a new 'Year' column by create a list of the current year using list comprehension
        census_pd['Year'] = [year for i in census_data]
#Convert FIPs codes tp state abbrevations
        State_abbrev_list = [FIPs_to_State_dict[i] for i in census_pd['STATE']]
#create a list of state names by converting state abbrevations to state names (using list comprehension)
        State_name_list = [reverse_us_state_abbrev[x] for x in State_abbrev_list]
#Essentially overwrite the 'STATE' column of the current census_pd with corresponding state name values
        census_pd['STATE'] = State_name_list
#Lastly append the current census_pd to our State_Pop_by_Year Datarame
        State_Pop_by_year = State_Pop_by_year.append(census_pd)
    except:
#If there is an exception, we simply go to the next year in the loop by using pass to end the exception
        pass

#Rename columns of our State_Pop_by_Year Datarame
State_Pop_by_year = State_Pop_by_year.rename(columns={'B01003_001E' :'Population Estimate', 'state' :'FIPs'})


In [93]:
#reindex our State_Pop_by_Year Datarame (since all indices were 0 for some reason)
State_Pop_by_year = State_Pop_by_year.reset_index()

#Drop the extra index column created by the reset_index method
State_Pop_by_year = State_Pop_by_year.drop(columns=['index'])

#Pivot the DataFrame 
Pop_df = State_Pop_by_year.pivot(index='Year', columns='STATE', values='Population Estimate')

#Clarify the meaning of each column by adding the word Population to the column names
for t in Pop_df.columns:
    Pop_df = Pop_df.rename(columns={t: t + ' Population'})
Pop_df

STATE,Alabama Population,Alaska Population,Arizona Population,Arkansas Population,California Population,Colorado Population,Connecticut Population,Delaware Population,Florida Population,Georgia Population,...,Tennessee Population,Texas Population,Utah Population,Vermont Population,Virginia Population,Washington Population,"Washington, DC Population",West Virginia Population,Wisconsin Population,Wyoming Population
Year,,,,,,,,,,,,,,,,,,,,,
2010,4712651.0,691189.0,6246816.0,2872684.0,36637290.0,4887061.0,3545837.0,881278.0,18511620.0,9468815.0,...,6234968.0,24311891.0,2657236.0,624258.0,7841754.0,6561297.0,584400.0,1840802.0,5637947.0,545579.0
2011,4747424.0,700703.0,6337373.0,2895928.0,36969200.0,4966061.0,3558172.0,890856.0,18688787.0,9600612.0,...,6297991.0,24774187.0,2715379.0,624958.0,7926192.0,6652845.0,593955.0,1846372.0,5664893.0,554697.0
2012,4777326.0,711139.0,6410979.0,2916372.0,37325068.0,5042853.0,3572213.0,900131.0,18885152.0,9714569.0,...,6353226.0,25208897.0,2766233.0,625498.0,8014955.0,6738714.0,605759.0,1850481.0,5687219.0,562803.0
2013,4799277.0,720316.0,6479703.0,2933369.0,37659181.0,5119329.0,3583561.0,908446.0,19091156.0,9810417.0,...,6402387.0,25639373.0,2813673.0,625904.0,8100653.0,6819579.0,619371.0,1853619.0,5706871.0,570134.0
2014,4817678.0,728300.0,6561516.0,2947036.0,38066920.0,5197580.0,3592053.0,917060.0,19361792.0,9907756.0,...,6451365.0,26092033.0,2858111.0,626358.0,8185131.0,6899123.0,633736.0,1853881.0,5724692.0,575251.0
2015,4830620.0,733375.0,6641928.0,2958208.0,38421464.0,5278906.0,3593222.0,926454.0,19645772.0,10006693.0,...,6499615.0,26538614.0,2903379.0,626604.0,8256630.0,6985464.0,647484.0,1851420.0,5742117.0,579679.0
2016,4841164.0,736855.0,6728577.0,2968472.0,38654206.0,5359295.0,3588570.0,934695.0,19934451.0,10099320.0,...,6548009.0,26956435.0,2948427.0,626249.0,8310301.0,7073146.0,659009.0,1846092.0,5754798.0,583029.0


In [94]:
#Reset index to have a column of years which will use to merge dataframes
new_Pop_df = Pop_df.reset_index()
new_Pop_df

STATE,Year,Alabama Population,Alaska Population,Arizona Population,Arkansas Population,California Population,Colorado Population,Connecticut Population,Delaware Population,Florida Population,...,Tennessee Population,Texas Population,Utah Population,Vermont Population,Virginia Population,Washington Population,"Washington, DC Population",West Virginia Population,Wisconsin Population,Wyoming Population
0,2010,4712651.0,691189.0,6246816.0,2872684.0,36637290.0,4887061.0,3545837.0,881278.0,18511620.0,...,6234968.0,24311891.0,2657236.0,624258.0,7841754.0,6561297.0,584400.0,1840802.0,5637947.0,545579.0
1,2011,4747424.0,700703.0,6337373.0,2895928.0,36969200.0,4966061.0,3558172.0,890856.0,18688787.0,...,6297991.0,24774187.0,2715379.0,624958.0,7926192.0,6652845.0,593955.0,1846372.0,5664893.0,554697.0
2,2012,4777326.0,711139.0,6410979.0,2916372.0,37325068.0,5042853.0,3572213.0,900131.0,18885152.0,...,6353226.0,25208897.0,2766233.0,625498.0,8014955.0,6738714.0,605759.0,1850481.0,5687219.0,562803.0
3,2013,4799277.0,720316.0,6479703.0,2933369.0,37659181.0,5119329.0,3583561.0,908446.0,19091156.0,...,6402387.0,25639373.0,2813673.0,625904.0,8100653.0,6819579.0,619371.0,1853619.0,5706871.0,570134.0
4,2014,4817678.0,728300.0,6561516.0,2947036.0,38066920.0,5197580.0,3592053.0,917060.0,19361792.0,...,6451365.0,26092033.0,2858111.0,626358.0,8185131.0,6899123.0,633736.0,1853881.0,5724692.0,575251.0
5,2015,4830620.0,733375.0,6641928.0,2958208.0,38421464.0,5278906.0,3593222.0,926454.0,19645772.0,...,6499615.0,26538614.0,2903379.0,626604.0,8256630.0,6985464.0,647484.0,1851420.0,5742117.0,579679.0
6,2016,4841164.0,736855.0,6728577.0,2968472.0,38654206.0,5359295.0,3588570.0,934695.0,19934451.0,...,6548009.0,26956435.0,2948427.0,626249.0,8310301.0,7073146.0,659009.0,1846092.0,5754798.0,583029.0


In [95]:
#From the merged_table (at the top) grab the 'Year', 'Year Average', and 'Value' columns
#then create a mini DataFrame
Poll_avg_df = merged_table[['Year', 'Year Average', 'Value']]
Poll_avg_df.head()

,Year,Year Average,Value
0,2000,0.150812,362.2790
1,2001,0.138810,361.3241
2,2002,0.136735,359.6222
3,2003,0.134243,355.1779
4,2004,0.122717,342.3452


In [96]:
#merge the Poll_avg_df and new_Pop_df on 'Year'
Merged_Pop_poll_df = pd.merge(Poll_avg_df, new_Pop_df, on="Year")

In [97]:
Merged_Pop_poll_df.head()

,Year,Year Average,Value,Alabama Population,Alaska Population,Arizona Population,Arkansas Population,California Population,Colorado Population,Connecticut Population,...,Tennessee Population,Texas Population,Utah Population,Vermont Population,Virginia Population,Washington Population,"Washington, DC Population",West Virginia Population,Wisconsin Population,Wyoming Population
0,2010,0.091656,306.2926,4712651.0,691189.0,6246816.0,2872684.0,36637290.0,4887061.0,3545837.0,...,6234968.0,24311891.0,2657236.0,624258.0,7841754.0,6561297.0,584400.0,1840802.0,5637947.0,545579.0
1,2011,0.087565,316.7458,4747424.0,700703.0,6337373.0,2895928.0,36969200.0,4966061.0,3558172.0,...,6297991.0,24774187.0,2715379.0,624958.0,7926192.0,6652845.0,593955.0,1846372.0,5664893.0,554697.0
2,2012,0.084872,298.2415,4777326.0,711139.0,6410979.0,2916372.0,37325068.0,5042853.0,3572213.0,...,6353226.0,25208897.0,2766233.0,625498.0,8014955.0,6738714.0,605759.0,1850481.0,5687219.0,562803.0
3,2013,0.078813,291.0552,4799277.0,720316.0,6479703.0,2933369.0,37659181.0,5119329.0,3583561.0,...,6402387.0,25639373.0,2813673.0,625904.0,8100653.0,6819579.0,619371.0,1853619.0,5706871.0,570134.0
4,2014,0.078189,277.7249,4817678.0,728300.0,6561516.0,2947036.0,38066920.0,5197580.0,3592053.0,...,6451365.0,26092033.0,2858111.0,626358.0,8185131.0,6899123.0,633736.0,1853881.0,5724692.0,575251.0


In [98]:
#Rename columns for clarification
renamed_Merged_Pop_poll_df = Merged_Pop_poll_df.rename(columns={'Year Average': 'Year Pullutant Average', 'Value': 'Year Mort Count'})
renamed_Merged_Pop_poll_df

,Year,Year Pullutant Average,Year Mort Count,Alabama Population,Alaska Population,Arizona Population,Arkansas Population,California Population,Colorado Population,Connecticut Population,...,Tennessee Population,Texas Population,Utah Population,Vermont Population,Virginia Population,Washington Population,"Washington, DC Population",West Virginia Population,Wisconsin Population,Wyoming Population
0,2010,0.091656,306.2926,4712651.0,691189.0,6246816.0,2872684.0,36637290.0,4887061.0,3545837.0,...,6234968.0,24311891.0,2657236.0,624258.0,7841754.0,6561297.0,584400.0,1840802.0,5637947.0,545579.0
1,2011,0.087565,316.7458,4747424.0,700703.0,6337373.0,2895928.0,36969200.0,4966061.0,3558172.0,...,6297991.0,24774187.0,2715379.0,624958.0,7926192.0,6652845.0,593955.0,1846372.0,5664893.0,554697.0
2,2012,0.084872,298.2415,4777326.0,711139.0,6410979.0,2916372.0,37325068.0,5042853.0,3572213.0,...,6353226.0,25208897.0,2766233.0,625498.0,8014955.0,6738714.0,605759.0,1850481.0,5687219.0,562803.0
3,2013,0.078813,291.0552,4799277.0,720316.0,6479703.0,2933369.0,37659181.0,5119329.0,3583561.0,...,6402387.0,25639373.0,2813673.0,625904.0,8100653.0,6819579.0,619371.0,1853619.0,5706871.0,570134.0
4,2014,0.078189,277.7249,4817678.0,728300.0,6561516.0,2947036.0,38066920.0,5197580.0,3592053.0,...,6451365.0,26092033.0,2858111.0,626358.0,8185131.0,6899123.0,633736.0,1853881.0,5724692.0,575251.0
5,2015,0.079513,276.3403,4830620.0,733375.0,6641928.0,2958208.0,38421464.0,5278906.0,3593222.0,...,6499615.0,26538614.0,2903379.0,626604.0,8256630.0,6985464.0,647484.0,1851420.0,5742117.0,579679.0
6,2016,0.081310,259.2433,4841164.0,736855.0,6728577.0,2968472.0,38654206.0,5359295.0,3588570.0,...,6548009.0,26956435.0,2948427.0,626249.0,8310301.0,7073146.0,659009.0,1846092.0,5754798.0,583029.0


In [99]:
#Create a new column for the estimated total US population for each year 
#Sum along columns for each row starting at the first state population column (indicated by the 3 in the iloc argument)
renamed_Merged_Pop_poll_df['Total US Population Estimate'] = renamed_Merged_Pop_poll_df.iloc[:, 3:].sum(axis=1)

In [100]:
renamed_Merged_Pop_poll_df

,Year,Year Pullutant Average,Year Mort Count,Alabama Population,Alaska Population,Arizona Population,Arkansas Population,California Population,Colorado Population,Connecticut Population,...,Texas Population,Utah Population,Vermont Population,Virginia Population,Washington Population,"Washington, DC Population",West Virginia Population,Wisconsin Population,Wyoming Population,Total US Population Estimate
0,2010,0.091656,306.2926,4712651.0,691189.0,6246816.0,2872684.0,36637290.0,4887061.0,3545837.0,...,24311891.0,2657236.0,624258.0,7841754.0,6561297.0,584400.0,1840802.0,5637947.0,545579.0,307727594.0
1,2011,0.087565,316.7458,4747424.0,700703.0,6337373.0,2895928.0,36969200.0,4966061.0,3558172.0,...,24774187.0,2715379.0,624958.0,7926192.0,6652845.0,593955.0,1846372.0,5664893.0,554697.0,310346358.0
2,2012,0.084872,298.2415,4777326.0,711139.0,6410979.0,2916372.0,37325068.0,5042853.0,3572213.0,...,25208897.0,2766233.0,625498.0,8014955.0,6738714.0,605759.0,1850481.0,5687219.0,562803.0,312855438.0
3,2013,0.078813,291.0552,4799277.0,720316.0,6479703.0,2933369.0,37659181.0,5119329.0,3583561.0,...,25639373.0,2813673.0,625904.0,8100653.0,6819579.0,619371.0,1853619.0,5706871.0,570134.0,315219560.0
4,2014,0.078189,277.7249,4817678.0,728300.0,6561516.0,2947036.0,38066920.0,5197580.0,3592053.0,...,26092033.0,2858111.0,626358.0,8185131.0,6899123.0,633736.0,1853881.0,5724692.0,575251.0,317746049.0
5,2015,0.079513,276.3403,4830620.0,733375.0,6641928.0,2958208.0,38421464.0,5278906.0,3593222.0,...,26538614.0,2903379.0,626604.0,8256630.0,6985464.0,647484.0,1851420.0,5742117.0,579679.0,320098094.0
6,2016,0.081310,259.2433,4841164.0,736855.0,6728577.0,2968472.0,38654206.0,5359295.0,3588570.0,...,26956435.0,2948427.0,626249.0,8310301.0,7073146.0,659009.0,1846092.0,5754798.0,583029.0,322087547.0


In [101]:
renamed_Merged_Pop_poll_df.to_csv("../DataFrames/State_Population.csv", header=True)